In [68]:
from sklearn.datasets import load_boston
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error

import pandas as pd
import numpy as np

In [69]:
# Gather Data
boston_dataset = load_boston()

data = pd.DataFrame(data=boston_dataset.data, columns=boston_dataset.feature_names)
features = data.drop(['INDUS', 'AGE'], axis=1)

log_prices = np.log(boston_dataset.target)
target = pd.DataFrame(log_prices, columns=['PRICE'])

C:\Users\migue\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function load_boston is deprecated; `load_boston` is deprecated in 1.0 and will be removed in 1.2.

    The Boston housing prices dataset has an ethical problem. You can refer to
    the documentation of this function for further details.

    The scikit-learn maintainers therefore strongly discourage the use of this
    dataset unless the purpose of the code is to study and educate about
    ethical issues in data science and machine learning.

    In this special case, you can fetch the dataset from the original
    source::

        import pandas as pd
        import numpy as np


        data_url = "http://lib.stat.cmu.edu/datasets/boston"
        raw_df = pd.read_csv(data_url, sep="\s+", skiprows=22, header=None)
        data = np.hstack([raw_df.values[::2, :], raw_df.values[1::2, :2]])
        target = raw_df.values[1::2, 2]

    Alternative datasets include the California housing dataset (

In [70]:
CRIME_IDX = 0
ZN_IDX = 1
CHAS_IDX = 2
RM_IDX = 4
PTRATIO_IDX = 8

# property_stats = np.ndarray(shape=(1, 11))
# property_stats[0][CRIME_IDX] = features['CRIM'].mean()
# property_stats[0][ZN_IDX] = features['ZN'].mean()
# property_stats[0][CHAS_IDX] = features['CHAS'].mean()

property_stats = features.mean().values.reshape(1, 11)

In [71]:
regr = LinearRegression().fit(features, target)
fitted_vals = regr.predict(features)

# Calculatin MSE & RMSE with sklearn, in log thousand dollars
MSE = round(mean_squared_error(y_true=target, y_pred=fitted_vals), 3) #default=True
RMSE = round(mean_squared_error(y_true=target, y_pred=fitted_vals, squared=False), 3) 

def get_log_estimate(nr_rooms, 
                     students_per_classroom, 
                     next_to_river=False, 
                     high_confidence=True):
    
    # Configure property
    property_stats [0] [RM_IDX] = nr_rooms
    property_stats [0] [PTRATIO_IDX] = students_per_classroom
    
    #
    if next_to_river:
        property_stats[0][CHAS_IDX] = 1
    else:
        property_stats[0][CHAS_IDX] = 0
        
    # Make prediction
    log_estimate = regr.predict(property_stats)[0][0]
    
    # Calc Range
    if high_confidence:
        upper_bound = log_estimate + 2*RMSE
        lower_bound = log_estimate - 2*RMSE
        interval = 95
    else:
        upper_bound = log_estimate + RMSE
        lower_bound = log_estimate - RMSE
        interval = 68
    
    return log_estimate, upper_bound, lower_bound, interval

In [72]:
get_log_estimate(3, 20, next_to_river = True, high_confidence=False)

C:\Users\migue\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(


(2.776758191480403, 2.9647581914804033, 2.588758191480403, 68)

In [74]:
np.median(boston_dataset.target)

21.2

In [81]:
# Converting log price estimates from 1970's including upper and lower bounds to today's prices.

ZILLOW_MEDIAN_PRICE = 583.3 # It's the median house price from the zillow real-state website
SCALE_FACTOR = ZILLOW_MEDIAN_PRICE / np.median(boston_dataset.target) # This would be equivalent to our interest rate

log_est, upper, lower, conf = get_log_estimate(9, students_per_classroom=15,
                                              next_to_river=False, high_confidence=False)
# Convert to today's dollars
dollar_est = np.e**log_est * 1000 * SCALE_FACTOR
dollar_hi = np.e**upper * 1000 * SCALE_FACTOR
dollar_low = np.e**lower * 1000 * SCALE_FACTOR

# Round the dollar values to nearest thousand
rounded_est = np.around(dollar_est, -3)
rounded_hi = np.around(dollar_hi, -3)
rounded_low = np.around(dollar_low, -3)

print(f'The estimated property values is {rounded_est}.')
print(f'At {conf}% confidence the valuation range is')
print(f'USD {rounded_low} at the lower end to USD {rounded_hi} at the high end.')

The estimated property values is 827000.0.
At 68% confidence the valuation range is
USD 685000.0 at the lower end to USD 998000.0 at the high end.


C:\Users\migue\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(


In [92]:
def get_dollar_estimate(rm, ptratio, chas=False, large_range=True):
    """ Estimate the price of a property in Boston
    
    Keyword arguments:
    rm -- number of rooms in the property
    ptratio -- number of students per teacher in the classroom for the school area
    chas -- True if the property is next to the river, False otherwise
    large_range -- True for a 95% prediction interval, false for a 68% interval
    
    """
    if rm < 1 or ptratio < 1:
        print('That is unrealistic. Try again.')
        return 
    
    log_est, upper, lower, conf = get_log_estimate(rm, 
                                                   ptratio, 
                                                   next_to_river=chas, 
                                                   high_confidence=large_range)
    # Convert to today's dollars
    dollar_est = np.e**log_est * 1000 * SCALE_FACTOR
    dollar_hi = np.e**upper * 1000 * SCALE_FACTOR
    dollar_low = np.e**lower * 1000 * SCALE_FACTOR

    # Round the dollar values to nearest thousand
    rounded_est = np.around(dollar_est, -3)
    rounded_hi = np.around(dollar_hi, -3)
    rounded_low = np.around(dollar_low, -3)

    print(f'The estimated property values is {rounded_est}.')
    print(f'At {conf}% confidence the valuation range is')
    print(f'USD {rounded_low} at the lower end to USD {rounded_hi} at the high end.')

In [90]:
get_dollar_estimate(rm=3, ptratio=20, chas=True)

The estimated property values is 442000.0.
At 95% confidence the valuation range is
USD 304000.0 at the lower end to USD 644000.0 at the high end.


C:\Users\migue\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(


In [ ]:
features.head()

In [ ]:
type(features.mean())

In [ ]:
type(features.mean().values)

In [97]:
import boston_valuation as val
val.get_dollar_estimate(6, 12, True)

The estimated property values is 783000.0.
At 95% confidence the valuation range is
USD 537000.0 at the lower end to USD 1140000.0 at the high end.


C:\Users\migue\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
